In [520]:
import nltk
import re
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from sklearn.metrics import classification_report

**Пробный текст:** Роберт Асприн "Другой отличный миф"

In [511]:
fd = open("text.txt", 'r', encoding='UTF-8')
text = fd.read()
print("..." + text[71:500] + "...")
fd.close()

...Д. Мартин.

Одно из немногих все искупляющих качеств наставников, думаю я, заключается в том, что их при случае можно одурачить. Это было верно, когда мать учила меня читать, это было верно, когда отец пытался научить меня быть фермером, и верно теперь, когда я обучаюсь магии.

– Ты не практиковался! – прервал мои размышления резкий упрек Гаркина.

– А вот и нет! – возразил я. – Это просто трудное упражнение.

Словно в ответ ...


### Ручная обработка текста

Разобью текст на предложения при помощи nltk

In [512]:
nltk.download('punkt')
sents = nltk.sent_tokenize(text)
for i in range(5000):
    print('(' + str(i) + ') ' + sents[i])

[nltk_data] Downloading package punkt to /home/valeria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(0) «Есть многое на свете, друг Горацио, что человеку знать не положено».
(1) Д. Мартин.
(2) Одно из немногих все искупляющих качеств наставников, думаю я, заключается в том, что их при случае можно одурачить.
(3) Это было верно, когда мать учила меня читать, это было верно, когда отец пытался научить меня быть фермером, и верно теперь, когда я обучаюсь магии.
(4) – Ты не практиковался!
(5) – прервал мои размышления резкий упрек Гаркина.
(6) – А вот и нет!
(7) – возразил я.
(8) – Это просто трудное упражнение.
(9) Словно в ответ левитируемое мною перо начало дрожать и колебаться в воздухе.
(10) – Ты не сосредотачиваешься!
(11) – обвинил он меня.
(12) – Это все ветер, – не согласился я. Мне хотелось добавить: «От твоего умного языка», но я не посмел.
(13) Гаркин в самом начале наших уроков продемонстрировал свое неумение ценить дерзких учеников.
(14) – Ветер!
(15) – презрительно фыркнул он, передразнивая меня.
(16) – Вот так надо, болван!
(17) Мой мысленный контакт с предметом моей соср

(3898) – Всего-навсего, да?
(3899) – скептически хмыкнул девол, достигая новых высот.
(3900) – Тебе отлично известно, что так не бывает.
(3901) Коль дракон привязался, то привязался навек.
(3902) Вот почему они так ценны.
(3903) – Навек?
(3904) – переспросил я.
(3905) – Ну...до тех пор, пока один из вас не умрет.
(3906) Но любой дурак знает, что дракона не кормят, если не хотят, если не хотят привязать его к себе.
(3907) Эти идиотские звери слишком впечатлительны, особенно такие молодые, как этот.
(3908) Я снова посмотрел на дракона.
(3909) Он был очень молод.
(3910) Крылья у него только начинали формироваться, что я воспринял, как признак незрелости, а клыки были острыми, как иглы, вместо изношенной закругленности концов у его собратьев по стойлу.
(3911) И все же под этой чешуей трепетала мускульная сила... Да, решил я, я смогу опереться на своего дракона в бою против любого...

– Глип!
(3912) – сказал дракон, облизывая раздвоенным языком оба конца своих усов.
(3913) Это привело меня 

Получилось неидеально, так как nltk разделил на предложения, где прямая речь оканчивается не запятой. Некоторые предложения не разделились.Многоточие не воспринимается как знак пунктуации. Исправим.

In [513]:
def correct_nltk(sents):
    corrected_sents = []
    i = 0
    while i < len(sents) - 1:
        div_error = re.search("(.+[.?!]) ([–А-Я0-9].+)", sents[i])
        tree_dots_ = re.search("(.+\.\.\.»?)(\n\n|\n\n\n\n  \n)((– )?[\.A-Я].+)", sents[i])
        chapter = re.search("(ГЛАВА [0-9][0-9]?)\n\n(«.+)", sents[i])
        if re.search("^– [а-я0-9]", sents[i + 1]) is not None:
            corrected_sents.append(sents[i] + ' ' + sents[i + 1])
            i += 1
        elif tree_dots_ is not None:
            corrected_sents.append(tree_dots_.group(1))
            corrected_sents.append(tree_dots_.group(3))
        elif div_error is not None:
            if len(div_error.group(1)) != 2:                # в этом тексте инициалы отделены пробелом,
                corrected_sents.append(div_error.group(1))  # нужен костыль для предотвращения 
                corrected_sents.append(div_error.group(2))  # их выделения в отдельные предложения
            else:
                corrected_sents.append(sents[i])
        elif chapter is not None:
            corrected_sents.append(chapter.group(1))
            corrected_sents.append(chapter.group(2))
        else:
            corrected_sents.append(sents[i])
        i += 1
    return corrected_sents


# так как в одном ошибочном разбиении может оказаться несколько предложений,
# функцию нужно повторить пару раз
for i in range(2):
    corrected_sents = correct_nltk(sents)
    sents = corrected_sents

In [514]:
for i in range(5000):
    print('(' + str(i) + ') ' + corrected_sents[i])

(0) «Есть многое на свете, друг Горацио, что человеку знать не положено».
(1) Д. Мартин.
(2) Одно из немногих все искупляющих качеств наставников, думаю я, заключается в том, что их при случае можно одурачить.
(3) Это было верно, когда мать учила меня читать, это было верно, когда отец пытался научить меня быть фермером, и верно теперь, когда я обучаюсь магии.
(4) – Ты не практиковался!
(5) – прервал мои размышления резкий упрек Гаркина.
(6) – А вот и нет!
(7) – возразил я.
(8) – Это просто трудное упражнение.
(9) Словно в ответ левитируемое мною перо начало дрожать и колебаться в воздухе.
(10) – Ты не сосредотачиваешься!
(11) – обвинил он меня.
(12) – Это все ветер, – не согласился я.
(13) Мне хотелось добавить: «От твоего умного языка», но я не посмел.
(14) Гаркин в самом начале наших уроков продемонстрировал свое неумение ценить дерзких учеников.
(15) – Ветер!
(16) – презрительно фыркнул он, передразнивая меня.
(17) – Вот так надо, болван!
(18) Мой мысленный контакт с предметом моей

(2386) – Из воздуха!
(2387) Бpось, малыш, ты же маг, а не элементал.
(2388) – А что такое элементал?
(2389) – Забудь об этом.
(2390) Я хочу сказать, что ты не пpивязан к какому-либо из четыpех элементов.
(2391) Ты – маг.
(2392) Ты упpавляешь ими, или по кpайне меpе, влияешь на них и чеpпаешь из них свою энеpгию.
(2393) Когда ты летаешь, то все, что тебе нужно, это чеpпать энеpгию из воздуха вместо земли.
(2394) – Если ты так утвеpждаешь, Ааз, – с сомнением сказал я.
(2395) – Отлично.
(2396) Сначала отыщи силовую линию.
(2397) – Но ведь мы покинули ее, когда свеpнули пpоведать девола, – сказал я.
(2398) – Малыш, силовых линий великое множество.
(2399) Одно лишь то, что мы покинули какую-то наземную силовую линию, не означает, что мы совеpшенно вне контакта с ней.
(2400) Пpовеpь-ка, нет ли силовой линии в воздухе?
(2401) – В воздухе?
(2402) – Поверь мне малыш.
(2403) Поищи.
(2404) Я вздохнул и закрыл глаза.
(2405) Обратив лицо к небу, я пытался представить обоюдоострое копье.
(2406) Спер

(4885) – Ты знаешь, кто я?
(4886) – удивленно спросил я.
(4887) – Я знаю все о тебе и твоих друзьях.
(4888) Я все время знал, что вы пытаетесь сделать.
(4889) Я попробовал установить вокруг себя полог, но не смог найти силовой линии.
(4890) Я хотел убежать, но прирос к месту.
(4891) – Видишь, насколько мои способности превосходят твои?
(4892) А ты еще хотел бросить мне вызов.
(4893) Я попытался побороть волну отчаянья.
(4894) – Подожди, пока придут другие!
(4895) – вызывающе крикнул я.
(4896) – Они уже пришли!
(4897) – прогремел голос.
(4898) – Смотри!
(4899) Из темноты ко мне выкатились два предмета.
(4900) Я с ужасом увидел, что это головы!
(4901) Танды и Квингли!
(4902) Я почувствовал дурноту, но цеплялся за обрывок надежды.
(4903) Не было еще никаких признаков Ааза.
(4904) Если он все еще на свободе, то, возможно, мы...
(4905) – Не жди помощи от извращенца, – ответил на мои мысли голос.
(4906) – Я с ним тоже разделался.
(4907) Появился охваченный огнем Ааз.
(4908) Он сделал, шатаяс

Берем из corrected_sents первые 100 предложений, сохраняем. Бьем текст с помощью регулярок и проверяем качество. Придется отрезать в конце каждой строки эталона знак препинания, иначе из-за сплита в в тесте и в эталоне ни одно предожение не будет полностью совпадать.

In [521]:
metric = []
for i in range(100):
    metric.append(corrected_sents[i][:-1])

reg_test = re.split('[.?!]', text)
i = 0
while reg_test[i].find(metric[99]) == -1:
    i += 1
reg_test = reg_test[:i + 1]

print("Accuracy: " + str(len(set(metric) & set(reg_test)) / len(set(metric) | set(reg_test))))

Accuracy: 0.004878048780487805


Качество очень плохое, так как сплит сохранил все переносы строк - из-за этого многие строки не похожи, хотя и разбиты верно.

### Обучение модели

In [516]:
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [517]:
# Тут придется выделить кусок заново, чтобы сохранить в нем пунктуацию.
metric = corrected_sents[:100]
test = tokenizer.tokenize(text)
i = 0
while test[i].find(metric[99]) == -1:
    i += 1
test = test[:i + 1]
test

['«Есть многое на свете, друг Горацио, что человеку знать не положено».',
 'Д. Мартин.',
 'Одно из немногих все искупляющих качеств наставников, думаю я, заключается в том, что их при случае можно одурачить.',
 'Это было верно, когда мать учила меня читать, это было верно, когда отец пытался научить меня быть фермером, и верно теперь, когда я обучаюсь магии.',
 '– Ты не практиковался!',
 '– прервал мои размышления резкий упрек Гаркина.',
 '– А вот и нет!',
 '– возразил я.',
 '– Это просто трудное упражнение.',
 'Словно в ответ левитируемое мною перо начало дрожать и колебаться в воздухе.',
 '– Ты не сосредотачиваешься!',
 '– обвинил он меня.',
 '– Это все ветер, – не согласился я.',
 'Мне хотелось добавить: «От твоего умного языка», но я не посмел.',
 'Гаркин в самом начале наших уроков продемонстрировал свое неумение ценить дерзких учеников.',
 '– Ветер!',
 '– презрительно фыркнул он, передразнивая меня.',
 '– Вот так надо, болван!',
 'Мой мысленный контакт с предметом моей сосредоточ

In [505]:
print("Accuracy: " + str(len(set(metric) & set(test)) / len(set(metric) | set(test))))

Accuracy: 0.9405940594059405


Качество очень хорошее, а ошибки у этого алгоритма те же, что мне пришлось исправлять вручную в коде выше. Сокращениями качество не повысить (здесь есть только одно сокращение, и оно учтено верно), но в порядке развлечения попробую добавить что-то в список сокращений.

In [518]:
print(tokenizer._params.abbrev_types)
tokenizer._params.abbrev_types.add('Д')
print(tokenizer._params.abbrev_types)

{'см', 'д', 'ч', 'ф', 'б', 'п.т', 'дж', 'рот'}
{'см', 'д', 'ч', 'ф', 'б', 'п.т', 'дж', 'Д', 'рот'}
